# Stream Cipher

A stream cipher is a symmetric key cipher where the plaintext is encrypted (and ciphertext is decrypted) one digit at a time. A digit usually is either a bit or a byte.

![stream-cipher](images/stream-cipher.png)

Encryption (decryption) is achieved by xoring the plaintext (ciphertext) with a stream of pseudorandom digits obtained as an expansion of the key.

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

from itertools import islice
from functools import reduce
from operator import xor

We import the handy function and classes implemented in the previous notebook and collected in a **module** named `lfsr.py`.

In [2]:
from lfsr import LFSR, reverse, bits_to_int, int_to_bits

## Pseudo Random Byte Generator

Although a stream cipher can work at bit level (each digit is a bit), usually stream ciphers are used considering one byte at a time (each digit is a byte). That is the reason we need an object to transform pseudo random bit generators, such as LFSRs, into pseudo random byte generators.

**Inputs**:
- **seed**: (optional, default `None`) initial state (`int`) representing the shared secret key.
- **bit_generator**: (optional, default `None`) Iterable that yields a pseudo-random bit starting from an initial seed. If None an LFSR is employed.

**Yield**:
- `byte`: pseudo-random byte (`int`)

### Implementation as Generator

In [3]:
def pseudo_random_byte_generator(seed=None, bit_generator=None, **kwargs):
    '''
    Parameters
    ----------
    seed: int or bytes, optional (default None)
        Seed for the pseudo random bit generator.
        If None, the value 0 is adopted
    bit_generator: iterator, optional (default None)
        Pseudo random bit generator. It must must takes seed (int) as first 
        positional arguments and return an iterable that yields a bit.
        If None, an LFSR with feedback polynomial x^12+x^6+x^4+x+1 is 
        adopted.
    kwargs: mapping, optional
        a dictionary of keyword arguments passed into `bit_generator`.
    Yield
    -----
    byte: int,
        pseudo random number in the set {0, 1, ..., 255}
    '''
    if seed is None:
        seed = 0
    if isinstance(seed, bytes):
        seed = int.from_bytes(seed, byteorder='little')
    if not isinstance(seed, int):
        raise TypeError(f'Unsupported type {type(seed)} for seed')

    if bit_generator is None:
        poly = [12, 6, 4, 1, 0]
        bits = LFSR(poly, state=0xFFF if seed == 0 else seed)
    else:
        if seed is None:
            seed = 0
        bits = bit_generator(seed, **kwargs)
    
    while True:
        byte = bits_to_int([bit for bit in islice(bits, 8)])
        yield byte

In [4]:
num_digits = 2000
prbg = pseudo_random_byte_generator()
digits = [byte for byte in islice(prbg, num_digits)]
bytes(digits)[:100]

b'\xff\xd7\xa2\x1aK\xed\xa0\xcb\xd3\xbc\xd8\x7f\xf3S\xfa#C\x82+\x90\xfa\xe7W:\x18<k\x95\x99\x1dE|t\x1b\x90\xcb\xe2\xb96s\x98\xf7f\xd0\n\x95\xfb\x17\x99e\xa2S\xa9,q\x97\x96|_\xf3\x8d\x03h\xb4\xcf\x13\x87\xa7\x96MZ\x1d\x81h\xcc(<Z\x90w\x11.\xd8\xe8\xe8\xd9\xdc2>U\xf1m\x9e\xd7\xc0\x10\x97\xf4v\x83'

### Implementation as Iterator

In [5]:
class PseudoRandomByteGenerator():
    '''
    Class implementing a pseudo random byte generator starting from a pseudo
    random bit generator.
    '''
    def __init__(self, seed=None, bit_generator=None, **kwargs):
        '''
        Parameters
        ----------
        seed: int or bytes, optional (default None)
            Seed for the pseudo random bit generator.
            If None, the value 0 is adopted
        bit_generator: iterator, optional (default None)
            Pseudo random bit generator. It must must takes seed (int) as first 
            positional arguments and return an iterable that yields a bit.
            If None, an LFSR with feedback polynomial x^12+x^6+x^4+x+1 is 
            adopted.
        kwargs: mapping, optional
            a dictionary of keyword arguments passed into `bit_generator`.
        '''
        if seed is None:
            seed = 0
        if isinstance(seed, bytes):
            seed = int.from_bytes(seed, byteorder='little')
        if not isinstance(seed, int):
            raise TypeError(f'Unsupported type {type(seed)} for seed')
            
        if bit_generator is None:
            poly = [12, 6, 4, 1, 0]
            if seed == 0:
                seed = None
            self.bit_generator = LFSR(poly, state=seed)
        else:
            self.bit_generator = bit_generator(seed, **kwargs)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        '''Generate a Pseudo Random Byte (int)'''
        byte = bits_to_int([bit for bit in islice(self.bit_generator, 8)])
        return byte

In [6]:
num_digits = 2000
prbg = PseudoRandomByteGenerator()
digits = [byte for byte in islice(prbg, num_digits)]
bytes(digits)[:100]

b'\xff\xd7\xa2\x1aK\xed\xa0\xcb\xd3\xbc\xd8\x7f\xf3S\xfa#C\x82+\x90\xfa\xe7W:\x18<k\x95\x99\x1dE|t\x1b\x90\xcb\xe2\xb96s\x98\xf7f\xd0\n\x95\xfb\x17\x99e\xa2S\xa9,q\x97\x96|_\xf3\x8d\x03h\xb4\xcf\x13\x87\xa7\x96MZ\x1d\x81h\xcc(<Z\x90w\x11.\xd8\xe8\xe8\xd9\xdc2>U\xf1m\x9e\xd7\xc0\x10\x97\xf4v\x83'

We expect bytes to be ditributed uniformely in the set of integers from 0 to 255, i.e., $\{0, 1, \dots, 255\}$.

In [7]:
B = 8
fig, ax = plt.subplots(figsize=(4,3))
ax.hist(list(digits), bins=np.arange(257), density=True)
ax.plot(np.array([0, 2**B-1]), (2**-B)*np.ones(2))
ax.grid(True)
ax.set(xlabel='byte', ylabel='probability')
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Stream Cipher

We define an object that, given a ky and a generator of pseudo random numbers in the set {0, ...,255} (all possible value for a byte), can encrypt and decrypt a message.

**Inputs**:
- **key**: `int` representing the shared secret key.
- **PRNG**: (optional, default None) Iterator implementing a PRNG that produce a pseudorandom bit stream starting from an initial seed. If None an LFSR is used as PRNG.

**Methods**:
- `encrypt`: encrypts a plaintext (`str` or `bytes`) and returns the corresponding cyphertext (`bytes`);
- `decrypt`: decrypts a cypertext (`str` or `bytes`) and returns the corresponding plaintext (`bytes`);

In [8]:
class StreamCipher(object):
    '''
    Stream cipher.

    Methods
    -------
    encrypt(self, plaintext):
        encrypt a plaintext
    decrypt(self, ciphertext): 
        decrypt a ciphertext
    '''
    def __init__(self, key, prng=None, **kwargs):        
        '''
        Parameters
        ----------
        key: int or bytes,
            secret key for PRNG initialization
        prng: iterator, optional (default None),
            pseudo random number generator (PRNG) for the generation of
            the random byte used for encryption and decryption
        kwargs: dict,
            keyword arguments for `prng`
        '''
        if prng is None:
            self.prng = PseudoRandomByteGenerator(key)
        else:
            self.prng = prng(key, **kwargs)
        
    def encrypt(self, plaintext):
        '''encrypt a `plaintext` (str, bytes) and  return the corresponding 
        cyphertext (bytes) '''
        return self._crypt(plaintext)
    
    def decrypt(self, ciphertext):
        ''' decrypt a `cypertext` (str, bytes) and return the corresponding 
        plaintext (bytes) '''
        return self._crypt(ciphertext)
    
    def _crypt(self, text):
        if isinstance(text, str):
            text = text.encode('utf-8')
        crypted = bytes([b^s for b, s in zip(text, self.prng)])
        return crypted
        

In [9]:
message = 'hello world!'
key = 0x0123456789ABCDEF

# create a StreamCipher instance for Alice and Bob
alice = StreamCipher(key) 
bob   = StreamCipher(key)

plaintextA = message.encode('utf-8')   # string to bytes 
ciphertext = alice.encrypt(plaintextA) # encryption by Alice
plaintextB = bob.decrypt(ciphertext)   # decryption by Bob

print(plaintextA) # -> b'hello world!' 
print(ciphertext) # -> b'U\xfe7\xaa\xc8\n\xc9\xa2\x81v|\xc3'
print(plaintextB) # -> b'hello world!' 


b'hello world!'
b'\x9f\x03\xbcf\xfa\xdb`\xf6\x17\xce7\x88'
b'hello world!'


## A5/1

A5/1 is a stream cipher used to provide privacy in the GSM cellular telephone standard.

It is based on a combination of three LFSRs with irregular clocking. At each cycle, the clocking bit of all three registers is examined and the majority bit is determined. A register is clocked if the clocking bit agrees with the majority bit.

| LFSR | length | &nbsp; &nbsp; Feedback Polynomial &nbsp; &nbsp; | clocking bit |
|:----:|:------:|:-----------------------------------------------:|:------------:|
|   1  |   19   |      \\( x^{19}+x^{18}+x^{17}+x^{14}+1 \\)      |      10      |
|   2  |   22   |      \\(               x^{22}+x^{21}+1 \\)      |      11      |
|   3  |   23   |      \\(    x^{23}+x^{22}+x^{21}+x^8+1 \\)      |      12      |

![A5/1](images/A5_1.png)


**Initialization**

A5/1 is initialised using a 64-bit private key together with a publicly known 22-bit frame number.
- Initially, the registers are set to zero. 
- The 64-bit secret key is mixed: in cycle $i$ (with 0≤$i$<64), the $i$-th key bit is added to the most significant bit of each register using XOR.
- Similarly, the 22-bits frame number is added in 22 cycles.

Then, the cipher is clocked using the normal majority clocking mechanism for 100 cycles, with the output discarded.

Finally, the cipher is ready to produce two 114 bit sequences of output keystream, first 114 for downlink, last 114 for uplink.

In [54]:

class A5_1(object):
    '''
    A5/1 stream cipher.
    
    Attributes
    ----------
    output: bool,
        stream cipher output bit.
    LFSRs: lfsr.LFSR,
        linear feedback shift registers that compose the A5/1 stream cipher
    majority: bool,
        majority bit
    '''
    
    def __init__(self, key, frame=0, verbose=False):
        '''
        key: int, list/string of 0/1, bytes, 
            64-bit key, bytes are considered little endian.
        frame: int or list/string of 0/1, 
            22-bit frame
        '''
        key_length = 64
        frame_length = 22
        polys = [
            [19, 18, 17, 14, 0],
            [22, 21, 0],
            [23, 22, 21, 8, 0],
        ]
                
        if isinstance(key, (bytes, bytearray)):
            key = int.from_bytes(key, byteorder='little')#wrong
        if isinstance(key, int):
            key = int_to_bits(key, key_length)
        elif (not hasattr(key, '__iter__')) \
            or (not len(key) == key_length) \
            or (not all(int(b) in (0, 1) for b in key)):
            raise TypeError('input type is not supported')
            
        if type(frame) is int:
            frame = int_to_bits(frame, frame_length)
        elif (not hasattr(frame, '__iter__')) \
            or (not len(frame) == frame_length) \
            or (not all(int(b) in (0, 1) for b in frame)):
            raise TypeError('input type is not supported')
        
        self.vebose = verbose
        self._LFSRs = [LFSR(poly, state=0) for poly in polys]
        self._ckbits = [10, 11, 12]
        self._count = 0
        
            
        # LFSRs initialization
        if verbose: 
            header = ' iter  LFSR1  LFSR2  LFSR3 maj out'
            print('--- key insertion ----')
            print(''.join([str(int(b)) for b in key]))
            print(header)
        for bit in key:
            self._insert_bit(bit)
        self._count = 0
            
        if verbose: 
            print('--- frame insertion ----')
            print(''.join([str(int(b)) for b in frame]))
            print(header)
        for bit in frame:
            self._insert_bit(bit)
        self._count = 0
            
        if verbose: print('--- key mixing ----')
        if verbose: print(header)
        for _ in range(100): 
            next(self)
        self._count = 0
            
        if verbose: print('--- stream cipher ----')
        if verbose: print(header)
            

    @property
    def LFSRs(self):
        return self._LFSRs
    
    @LFSRs.setter
    def LFSRs(self, val):
        raise AttributeError('Denied')

    @property
    def output(self):
        output = reduce(xor, [lfsr.output for lfsr in self.LFSRs])
        return output
    
    @output.setter
    def output(self, val):
        raise AttributeError('Denied')

    @property
    def majority(self):
        # get the values of ckbits for each LFSR
        bits = [bool(lfsr.state & (1 << ckbit))
               for lfsr, ckbit in zip(self.LFSRs, self._ckbits)]
        # compute majority
        majority = sum(bits) > (len(bits)//2)
        return majority
    
    @majority.setter
    def majority(self, val):
        raise AttributeError('Denied')
        
    def __iter__(self):
        return self
        
    def __next__(self):
        ''' clock stream cipher and returns the outputbit '''
        # clock LFSR_i if ckbit_i is equal to majority bit
        majority = self.majority
        for lfsr, ckbit in zip(self.LFSRs, self._ckbits):
            if bool(lfsr.state & (1 << ckbit)) == majority:
                next(lfsr)
        # update output and majority bit
        self._count += 1
        self._log()
        
        return self.output  
    
    def _insert_bit(self, bit):
        ''' Insert `bit` (bool) in each LFSR '''
        # every LFSR is clocked independently from majority bit
        for lfsr in self.LFSRs:
            lfsr.feedback ^= bit
            next(lfsr)
        # update output and majority bit
        self._count += 1
        self._log()
    
    def __str__(self):
        states = ', '.join([f'0x{lfsr.state:0{1+len(lfsr)//4}x}' 
                            for lfsr in self.LFSRs])
        _str = ', '.join([
            f'state: ({states})',
            f'majority: {int(self.majority)}',
            f'output: {int(self.output)}',
        ])
        return _str
        
    def __repr__(self):
        return f'A5/1({str(self)})'
    
    def _log(self):
        if self.vebose:
            _str = '  '.join([
                f'{self._count:5d}',
                ' '.join([f'{lfsr.state:0{1+len(lfsr)//4}x}' 
                          for lfsr in self.LFSRs]),
                f'{self.majority:2d}',
                f'{self.output:2d}',
            ])
            print(_str)
        

In [55]:
key, frame = 0x0123456789ABCDEF, 0x2F695A # my key
a51 = A5_1(key, frame=frame, verbose=True)

--- key insertion ----
1111011110110011110101011001000111100110101000101100010010000000
 iter  LFSR1  LFSR2  LFSR3 maj out
    1  40000 200000 400000   0   0
    2  60000 300000 600000   0   0
    3  70000 380000 700000   0   0
    4  78000 3c0000 780000   0   0
    5  3c000 1e0000 3c0000   0   0
    6  5e000 2f0000 5e0000   0   0
    7  6f000 378000 6f0000   0   0
    8  77800 3bc000 778000   0   0
    9  7bc00 3de000 3bc000   0   0
   10  3de00 1ef000 5de000   0   0
   11  5ef00 2f7800 2ef000   1   0
   12  6f780 37bc00 177800   1   0
   13  37bc0 1bde00 0bbc00   1   0
   14  1bde0 0def00 45de00   1   0
   15  0def0 26f780 22ef00   0   0
   16  06f78 337bc0 117780   1   0
   17  037bc 39bde0 48bbc0   1   0
   18  41bde 3cdef0 245de0   1   0
   19  20def 1e6f78 122ef0   1   1
   20  506f7 2f37bc 491778   1   1
   21  2837b 179bde 248bbc   0   1
   22  141bd 0bcdef 5245de   0   0
   23  4a0de 05e6f7 2922ef   0   0
   24  6506f 22f37b 149177   0   1
   25  72837 3179bd 4a48bb   0   1
  

In [17]:
message = 'hello world!'
key, frame = 0x0123456789ABCDEF, 0x2F695A # my key

# create a StreamCipher instance for both Alice and Bob
alice = StreamCipher(key, prng=PseudoRandomByteGenerator, bit_generator=A5_1, frame=frame)
bob = StreamCipher(key, prng=PseudoRandomByteGenerator, bit_generator=A5_1, frame=frame)

plaintextA = message.encode('utf-8')
ciphertext = alice.encrypt(plaintextA)
plaintextB = bob.decrypt(ciphertext)
print('plaintextA:', plaintextA)
print('ciphertext:', ciphertext)
print('plaintextB:', plaintextB)

plaintextA: b'hello world!'
ciphertext: b'\xec\xe3zM;@\x08\xf3r\xa0\x14\x1c'
plaintextB: b'hello world!'


In [27]:
message = 'a'
key, frame = 0x0123456789ABCDEF, 0x2F695A # my key

# create a StreamCipher instance for both Alice and Bob
alice = StreamCipher(
    key, 
    prng=PseudoRandomByteGenerator, 
    bit_generator=A5_1, 
    frame=frame,
    verbose=True,
)
bob = StreamCipher(
    key, 
    prng=PseudoRandomByteGenerator, 
    bit_generator=A5_1, 
    frame=frame
)

plaintextA = message.encode('utf-8')
ciphertext = alice.encrypt(plaintextA)
plaintextB = bob.decrypt(ciphertext)
print('plaintextA:', plaintextA)
print('ciphertext:', ciphertext)
print('plaintextB:', plaintextB)

--- key insertion ----
1111011110110011110101011001000111100110101000101100010010000000
 iter  LFSR1  LFSR2  LFSR3 maj out
    1  40000 200000 400000  0   0
    2  60000 300000 600000  0   0
    3  70000 380000 700000  0   0
    4  78000 3c0000 780000  0   0
    5  3c000 1e0000 3c0000  0   0
    6  5e000 2f0000 5e0000  0   0
    7  6f000 378000 6f0000  0   0
    8  77800 3bc000 778000  0   0
    9  7bc00 3de000 3bc000  0   0
   10  3de00 1ef000 5de000  0   0
   11  5ef00 2f7800 2ef000  1   0
   12  6f780 37bc00 177800  1   0
   13  37bc0 1bde00 0bbc00  1   0
   14  1bde0 0def00 45de00  1   0
   15  0def0 26f780 22ef00  0   0
   16  06f78 337bc0 117780  1   0
   17  037bc 39bde0 48bbc0  1   0
   18  41bde 3cdef0 245de0  1   0
   19  20def 1e6f78 122ef0  1   1
   20  506f7 2f37bc 491778  1   1
   21  2837b 179bde 248bbc  0   1
   22  141bd 0bcdef 5245de  0   0
   23  4a0de 05e6f7 2922ef  0   0
   24  6506f 22f37b 149177  0   1
   25  72837 3179bd 4a48bb  0   1
   26  3941b 38bcde 25245d 

In [26]:
print(alice.prng.bit_generator)

state: (0x578a8, 0x04cfd4, 0x785521), majority: 1, output: 1


## Rivest Cipher 4 (RC4)

RC4 is a stream cipher that generates the keystream from a secret internal state which consists of two parts:
- A permutation $P$ of all 256 possible bytes.
- Two 8-bit index-pointers (denoted $i$ and $j$).

$P$ is initialized with a variable length key by means of the *key-scheduling algorithm* (**KSA**).

Then, the keystream is generated using the *pseudo-random generation algorithm* (**PRGA**) that updates the indexes $i$ and $j$, modifies the permutation $P$ and generates a random byte.


**Key Scheduling Algorithm (KSA)**

The KSA is used to initialize the permutation $P$ starting from a key composed by 𝐿 bytes. Typical values for 𝐿 range from 40 to 256.

- $P$ is initialized with an identity permutation ($P[i]=i$).
- Then, bytes of $P$ are mixed iteratively in a way that depends on the key.

**Pseudocode**:
> **Input** \\( k = [k_0, k_1, \dots, k_{L-1}]\\), with \\( k_i \in {0, 1, \dots, 255} \\) <br>
> \\( j \leftarrow 0 \\) <br>
> **for** \\( i = 0, 1, \dots, 255 \\) <br>
> \\( \qquad P[i] \leftarrow i \\) <br>
> **endfor** <br>
> **for** \\( i = 0, 1, \dots, 255 \\) <br>
> \\( \qquad j \leftarrow (j + P[i] + k[i \\) mod \\( L]) \\) mod \\( 256 \\) <br>
> \\( \qquad P[i], P[j] \leftarrow P[j], P[i] \\) <br>
> **endfor** <br>
> **Output** \\( P \\) <br>

**Pseudo-random generation Algorithm (PRGA)**

For each iteration, PRGA modifies the state (represented by the permutation $P$ and the pair of indexes $i$, $j$) and outputs a byte.

In each iteration:
- $i$ is incremented, 
- $j$ is updated by adding the value $P[i]$, 
- $P[i]$ and $P[j]$ are swapped.
- the output byte is element of $P$ ant the location $P[i]+P[j]$  (mod 256)


**Pseudocode**:
> **State** \\( P, i, j \\) <br>
> \\( i \leftarrow (i + 1) \\) mod \\( 256 \\) <br>
> \\( j \leftarrow (j + P[i]) \\) mod \\( 256 \\) <br>
> \\(  P[i], P[j] \leftarrow P[j], P[i] \\) <br>
> \\( K \leftarrow P[(P[j] + P[i]) \\) mod \\( 256] \\) <br>
> **Output** \\( K \\) <br>

**RC4-drop[$n$]**

RC4 has many known vulnerabilities mainly related to the correlation between the key and the first bytes of the permutation $P$. 
Most of them can be avoided by discarding the first 𝑛 bytes of the output stream, from where it becomes RC4-drop[$n$].
Typical values for $n$ are:
- $n = 768$
- $n = 3072$ (more conservative value)


In [51]:
class RC4(object):
    '''
    Rivest Cipher 4 (RC4) Stream Cipher.
    
    Attributes
    ----------
    P: list of int,
        RC4 Permutation
    i, j: int,
        RC4 indices
    '''
    
    def __init__(self, key, key_length=None, drop=0):
        ''' 
        Parameters
        ----------
        key: int, bytes, str, list of int,
            secret key
        key_length: bytes, optional (default None)
            number of bytes composing the key in case key is an int.
        drop: int, optional (default 0)
            number of output bytes to drop after initialization
        '''    
        # check parameters
        if isinstance(key, int):
            key = int(key).to_bytes(byteorder='little', length=key_length)
        elif isinstance(key, (bytes, bytearray))\
            or (hasattr(key, '__iter__') \
                and all(isinstance(B, int) for B in key)):
            key = bytes(key)
        elif isinstance(key, str):
            key = key.encode('utf-8')
        else:
            raise TypeError('key format is not supported')
            
        if not isinstance(drop, int) or drop < 0:
            raise ValueError('drop must be an integer >= 0')
        
        # initialization
        self.P = self.ksa(key)
        self.i, self.j = 0, 0
        
        # dropping output bytes for safety issues
        for _ in range(drop):
            next(self)
    
    @staticmethod
    def ksa(key):
        ''' returns the permutation P initialized by means of the 
        Key-scheduling Algorithm (KSA) with the input key (bytes)
        '''
        P = list(range(256)) # identity permutation
        j = 0
        for i in range(256):
            j = (j + P[i] + key[i % len(key)]) % 256
            P[i], P[j] = P[j], P[i]
        return P
    
    def prga(self):
        ''' Pseudo-random generation algorithm (PRGA) that generates a random
        byte starting from the Permutation P and the pair of indexes i, j.
        '''
        self.i = (self.i + 1) % 256
        self.j = (self.j + self.P[self.i]) % 256
        self.P[self.i], self.P[self.j] = self.P[self.j], self.P[self.i]
        byte = self.P[(self.P[self.i] + self.P[self.j]) % 256]
        return byte
        
    def __iter__(self):
        return self
        
    def __next__(self):
        return self.prga() 
    
    def __str__(self):
        _str = ', '.join([
            f'i: 0x{self.i:02x}',
            f'j: 0x{self.j:02x}',
            f'P: {bytes(self.P)}',
        ])
        return _str
    
    def __repr__(self):
        return f'RC4({str(self)})'
        

In [52]:
message = 'hello world!'
key = b'0123456789ABCDEF'
ndrop = 3072
        
# create a StreamCipher instance for both Alice and Bob
alice = StreamCipher(key, prng=RC4, drop=ndrop) 
bob   = StreamCipher(key, prng=RC4, drop=ndrop)

plaintextA = message.encode('utf-8')   # string to bytes 
ciphertext = alice.encrypt(plaintextA) # encryption by Alice
plaintextB = bob.decrypt(ciphertext)   # decryption by Bob

print(plaintextA) # -> b'hello world!' 
print(ciphertext) # -> b'/\x9e\xf9\x83@\x81}\xa9\xd0\xd4\xd5\xf4' 
print(plaintextB) # -> b'hello world!' 


b'hello world!'
b'/\x9e\xf9\x83@\x81}\xa9\xd0\xd4\xd5\xf4'
b'hello world!'


In [53]:
print(alice.prng)

i: 0x0c, j: 0x1a, P: b'&\xc4\xe88\xea\xf8\x1f(DP\xc6\x19\xa0p\x9d\x84\xf3\xa7\xdb6a\x15\x0591\xad\xdaF\x87A0\xaf_w\n}Q>:\r\xd4\xf0s\xcaGg\x1b\xb3"\xc5\xddR\x18C`\xc0\x99\x0b\xdffm\xbf\x8c\x93\xce=\x04\xb9\xac\x88@\xa97E\x0c\xf7\xd2X\xc8\xa8?\xc7\xd9T\x8d\t\x8bK\xcd\xdc\xef\xcc\xb0\x9fnI\x83N\xe4\x17\x1dx \x10\xcb\x03\x13\x0f~!\xb5\xae\xfat+[\xee\xcfq\x16\xfdS\xd5\xe2\xb7\x80\x9e\xc1iOBU\x95\x06\xe92\x9b\xe3h-\x12\xc9\x11\x14\xa1\x8a\xf2\xa3%z\x94\x07r\xd3H\xb4yL.\'\x1c\x7f)\xc2\xbab$\xec\xed\xa5\x82\xd8\x90\x1a\xfe\xfbMu\x0e\x89jV\xde\xe7<5]J*\xaa\x96\xf6d\xbb\x9c\xb23\x00\xff\xf1\xeb\xd0\xf9\x01W\xe0\x81l#\x97c\xfc\xe1oe\xd7^\xf4\\\xf5\x9a\x08\x85Y,\xbcZ\xab\xa6\xbd\x98\xb14\xbe|\xd1\xc3\x86\xe6v/\xb8k\x8e\xb6\xe5\x92\x1e{\xa2;\x02\x8f\xd6\x91\xa4'
